To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3 8b is trained on a crazy 15 trillion tokens! Llama-2 was 2 trillion.**

Use our [Llama-3 8b Instruct](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing) notebook for conversational style finetunes.

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes # Downgrade xformers to a version below 0.0.27

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/My Drive/template_experiment/mistral/

/content/drive/My Drive/template_experiment/mistral


In [9]:
#if train and validation exist
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset

dataset = load_dataset("robbaniirfan/fallacy_template", split='train')
train_df = dataset.to_pandas()

train_df, validation_df = train_test_split(train_df, test_size=0.25, stratify=train_df['fallacy'])
validation_split = Dataset.from_pandas(validation_df)
train_split = Dataset.from_pandas(train_df)
#validation_df.to_csv("validation.csv", index=False)
#train_df.to_csv("train.csv", index=False)

Generating train split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
#if train and validation exist
import pandas as pd
from datasets import load_dataset, Dataset

train_df = pd.read_csv("train.csv")
validation_df = pd.read_csv("validation.csv")
#train_split = Dataset.from_pandas(train_df)
#validation_split = Dataset.from_pandas(validation_df)

In [11]:
alpaca_prompt = """You are a master at identifying the structure of logic fallacies

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = train_split.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/150 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.5 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 150 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.274500
2,1.214500
3,1.240400
4,1.147100
5,1.022500
6,0.809900
7,0.622500
8,0.618200
9,0.398900
10,0.294600


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

803.035 seconds used for training.
13.38 minutes used for training.
Peak reserved memory = 6.191 GB.
Peak reserved memory for training = 1.691 GB.
Peak reserved memory % of max memory = 41.979 %.
Peak reserved memory for training % of max memory = 11.466 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "# Task\nIdentify the underlying structure of a fallacious argument which is a False Dilemma.\nGiven a list of fallacy templates, your task is to choose a template that best describes the underlying fallacy structure, choosing the template's placeholders, [A] and [C], directly from the input text. Additionally, the text must be a consecutive sequence of one or more terms without any conjugation. Please follow the output format.\n\n# Definitions\nEntity: a noun phrase in the input.\nEvent: a verb phrase in the input.\nPlaceholder: A fill-in-the-blank choice within a template. Each placeholder may either be an entity or an event.\nPlease note! Placeholders can ONLY be either an entity (i.e., noun phrase) or an event (i.e., verb phrase) and may not be any other type of phrase (e.g., prepositional phrase).\n\n# List of Templates\nTemplate No.1:\nPremise 1: An entity/event [A] promotes a good entity/event [C].\nPremise 2: The absence of an entity/event [A] suppresses a good entity/event [C].\nConclusion: Therefore, [A] should be brought about.\n\nTemplate No.2:\nPremise 1: An entity/event [A] suppresses a bad entity/event [C]\nPremise 2: The absence of an entity/event [A] promotes a bad entity/event [C].\nConclusion: Therefore, [A] should be brought about.\n\nTemplate No.3:\nPremise 1: An entity/event [A] suppresses a good entity/event [C]\nPremise 2: The absence of an entity/event [A] promotes a good entity/event [C].\nConclusion: Therefore, [A] should not be brought about.\n\nTemplate No.4:\nPremise 1: An entity/event [A] promotes a bad entity/event [C]\nPremise 2: The absence of an entity/event [A] suppresses a bad entity/event [C].\nConclusion: Therefore, [A] should not be brought about.\n\nTemplate No.5:\nThe argument cannot be instantiated with one of the templates above.\n\n# Output Format\nTemplate No.=[No.]\n[A]=\n[C]=\n\n# Important Criteria: Prioritizing entities over events for placeholder.\nFor choosing placeholder, please prioritize entities over events in the case that an entity itself captures the underlying intent of the argument opposed to the event. However, if the event makes more sense, please choose an event for the placeholder.\n\n# Correct Example\nInput: To get better schools, we have to raise taxes. If we don't, we can't have better schools.\nOutput:\nTemplate No.=1\n[A]=raise taxes\n[C]=schools\nExplanation:\nHere, there are 2 possible options for [C] which are \"schools\" (i.e., entity) and \"can't have better schools\" (i.e., event). Since the entity is the top priority and the second option does not work with template 1 because it is a suppressed relation, \"schools\" is cchosen for [C].\nAlso, [A] and [C] are taken directly from the input text. For example, \"raising taxes\" as [A] also sounds correct, but the term \"raising\" is not mentioned in the input text. That is why \"raise taxes\" is chosen for [A]. Because the argument believes that \"raise taxes\" promote \"schools\" while not \"raise taxes\" suppress \"school\". he conclusion is implicit that \u201cPremise 1 supports that raise taxes should be brought about.\u201d Thus, Template No.=1 is selected.\n\n# Wrong Example\nInput: To get better schools, we have to raise taxes. If we don't, we can't have better schools.\nOutput:\nTemplate No.=1\n[A]=raising taxes\n[C]=can't have better schools\nExplanation:\nHere, there are 2 possible options for [C] which are \"schools\" (i.e., entity) and \"can't have better schools\" (i.e., event). However, \"can't have better schools\" as [C] is incorrect because it is an event instead of the entity of \"schools\" which already makes sense.\nAlso, \"raising taxes\" as [A] is incorrect because the placeholder is not taken directly from the text. Here \"raising taxes\" is chosen as [A] but the word \"raising\" does not appear in the input text. Therefore the correct correct choice for [A] is \"raise taxes\".\n\nAgain, please only select the placeholders directly from the text!\n", # instruction
        "We either have to cut taxes or leave a huge debt for our children.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> You are a master at identifying the structure of logic fallacies\n\n### Instruction:\n# Task\nIdentify the underlying structure of a fallacious argument which is a False Dilemma.\nGiven a list of fallacy templates, your task is to choose a template that best describes the underlying fallacy structure, choosing the template\'s placeholders, [A] and [C], directly from the input text. Additionally, the text must be a consecutive sequence of one or more terms without any conjugation. Please follow the output format.\n\n# Definitions\nEntity: a noun phrase in the input.\nEvent: a verb phrase in the input.\nPlaceholder: A fill-in-the-blank choice within a template. Each placeholder may either be an entity or an event.\nPlease note! Placeholders can ONLY be either an entity (i.e., noun phrase) or an event (i.e., verb phrase) and may not be any other type of phrase (e.g., prepositional phrase).\n\n# List of Templates\nTemplate No.1:\nPremise 1: An entity/event [A] promotes a good entity/

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "# Task\nIdentify the underlying structure of a fallacious argument which is a False Dilemma.\nGiven a list of fallacy templates, your task is to choose a template that best describes the underlying fallacy structure, choosing the template's placeholders, [A] and [C], directly from the input text. Additionally, the text must be a consecutive sequence of one or more terms without any conjugation. Please follow the output format.\n\n# Definitions\nEntity: a noun phrase in the input.\nEvent: a verb phrase in the input.\nPlaceholder: A fill-in-the-blank choice within a template. Each placeholder may either be an entity or an event.\nPlease note! Placeholders can ONLY be either an entity (i.e., noun phrase) or an event (i.e., verb phrase) and may not be any other type of phrase (e.g., prepositional phrase).\n\n# List of Templates\nTemplate No.1:\nPremise 1: An entity/event [A] promotes a good entity/event [C].\nPremise 2: The absence of an entity/event [A] suppresses a good entity/event [C].\nConclusion: Therefore, [A] should be brought about.\n\nTemplate No.2:\nPremise 1: An entity/event [A] suppresses a bad entity/event [C]\nPremise 2: The absence of an entity/event [A] promotes a bad entity/event [C].\nConclusion: Therefore, [A] should be brought about.\n\nTemplate No.3:\nPremise 1: An entity/event [A] suppresses a good entity/event [C]\nPremise 2: The absence of an entity/event [A] promotes a good entity/event [C].\nConclusion: Therefore, [A] should not be brought about.\n\nTemplate No.4:\nPremise 1: An entity/event [A] promotes a bad entity/event [C]\nPremise 2: The absence of an entity/event [A] suppresses a bad entity/event [C].\nConclusion: Therefore, [A] should not be brought about.\n\nTemplate No.5:\nThe argument cannot be instantiated with one of the templates above.\n\n# Output Format\nTemplate No.=[No.]\n[A]=\n[C]=\n\n# Important Criteria: Prioritizing entities over events for placeholder.\nFor choosing placeholder, please prioritize entities over events in the case that an entity itself captures the underlying intent of the argument opposed to the event. However, if the event makes more sense, please choose an event for the placeholder.\n\n# Correct Example\nInput: To get better schools, we have to raise taxes. If we don't, we can't have better schools.\nOutput:\nTemplate No.=1\n[A]=raise taxes\n[C]=schools\nExplanation:\nHere, there are 2 possible options for [C] which are \"schools\" (i.e., entity) and \"can't have better schools\" (i.e., event). Since the entity is the top priority and the second option does not work with template 1 because it is a suppressed relation, \"schools\" is cchosen for [C].\nAlso, [A] and [C] are taken directly from the input text. For example, \"raising taxes\" as [A] also sounds correct, but the term \"raising\" is not mentioned in the input text. That is why \"raise taxes\" is chosen for [A]. Because the argument believes that \"raise taxes\" promote \"schools\" while not \"raise taxes\" suppress \"school\". he conclusion is implicit that \u201cPremise 1 supports that raise taxes should be brought about.\u201d Thus, Template No.=1 is selected.\n\n# Wrong Example\nInput: To get better schools, we have to raise taxes. If we don't, we can't have better schools.\nOutput:\nTemplate No.=1\n[A]=raising taxes\n[C]=can't have better schools\nExplanation:\nHere, there are 2 possible options for [C] which are \"schools\" (i.e., entity) and \"can't have better schools\" (i.e., event). However, \"can't have better schools\" as [C] is incorrect because it is an event instead of the entity of \"schools\" which already makes sense.\nAlso, \"raising taxes\" as [A] is incorrect because the placeholder is not taken directly from the text. Here \"raising taxes\" is chosen as [A] but the word \"raising\" does not appear in the input text. Therefore the correct correct choice for [A] is \"raise taxes\".\n\nAgain, please only select the placeholders directly from the text!\n", # instruction
        "We either have to cut taxes or leave a huge debt for our children.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a master at identifying the structure of logic fallacies

### Instruction:
# Task
Identify the underlying structure of a fallacious argument which is a False Dilemma.
Given a list of fallacy templates, your task is to choose a template that best describes the underlying fallacy structure, choosing the template's placeholders, [A] and [C], directly from the input text. Additionally, the text must be a consecutive sequence of one or more terms without any conjugation. Please follow the output format.

# Definitions
Entity: a noun phrase in the input.
Event: a verb phrase in the input.
Placeholder: A fill-in-the-blank choice within a template. Each placeholder may either be an entity or an event.
Please note! Placeholders can ONLY be either an entity (i.e., noun phrase) or an event (i.e., verb phrase) and may not be any other type of phrase (e.g., prepositional phrase).

# List of Templates
Template No.1:
Premise 1: An entity/event [A] promotes a good entity/event [C].
Premise

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [12]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "# Task\nIdentify the underlying structure of a fallacious argument which is a False Dilemma.\nGiven a list of fallacy templates, your task is to choose a template that best describes the underlying fallacy structure, choosing the template's placeholders, [A] and [C], directly from the input text. Additionally, the text must be a consecutive sequence of one or more terms without any conjugation. Please follow the output format.\n\n# Definitions\nEntity: a noun phrase in the input.\nEvent: a verb phrase in the input.\nPlaceholder: A fill-in-the-blank choice within a template. Each placeholder may either be an entity or an event.\nPlease note! Placeholders can ONLY be either an entity (i.e., noun phrase) or an event (i.e., verb phrase) and may not be any other type of phrase (e.g., prepositional phrase).\n\n# List of Templates\nTemplate No.1:\nPremise 1: An entity/event [A] promotes a good entity/event [C].\nPremise 2: The absence of an entity/event [A] suppresses a good entity/event [C].\nConclusion: Therefore, [A] should be brought about.\n\nTemplate No.2:\nPremise 1: An entity/event [A] suppresses a bad entity/event [C]\nPremise 2: The absence of an entity/event [A] promotes a bad entity/event [C].\nConclusion: Therefore, [A] should be brought about.\n\nTemplate No.3:\nPremise 1: An entity/event [A] suppresses a good entity/event [C]\nPremise 2: The absence of an entity/event [A] promotes a good entity/event [C].\nConclusion: Therefore, [A] should not be brought about.\n\nTemplate No.4:\nPremise 1: An entity/event [A] promotes a bad entity/event [C]\nPremise 2: The absence of an entity/event [A] suppresses a bad entity/event [C].\nConclusion: Therefore, [A] should not be brought about.\n\nTemplate No.5:\nThe argument cannot be instantiated with one of the templates above.\n\n# Output Format\nTemplate No.=[No.]\n[A]=\n[C]=\n\n# Important Criteria: Prioritizing entities over events for placeholder.\nFor choosing placeholder, please prioritize entities over events in the case that an entity itself captures the underlying intent of the argument opposed to the event. However, if the event makes more sense, please choose an event for the placeholder.\n\n# Correct Example\nInput: To get better schools, we have to raise taxes. If we don't, we can't have better schools.\nOutput:\nTemplate No.=1\n[A]=raise taxes\n[C]=schools\nExplanation:\nHere, there are 2 possible options for [C] which are \"schools\" (i.e., entity) and \"can't have better schools\" (i.e., event). Since the entity is the top priority and the second option does not work with template 1 because it is a suppressed relation, \"schools\" is cchosen for [C].\nAlso, [A] and [C] are taken directly from the input text. For example, \"raising taxes\" as [A] also sounds correct, but the term \"raising\" is not mentioned in the input text. That is why \"raise taxes\" is chosen for [A]. Because the argument believes that \"raise taxes\" promote \"schools\" while not \"raise taxes\" suppress \"school\". he conclusion is implicit that \u201cPremise 1 supports that raise taxes should be brought about.\u201d Thus, Template No.=1 is selected.\n\n# Wrong Example\nInput: To get better schools, we have to raise taxes. If we don't, we can't have better schools.\nOutput:\nTemplate No.=1\n[A]=raising taxes\n[C]=can't have better schools\nExplanation:\nHere, there are 2 possible options for [C] which are \"schools\" (i.e., entity) and \"can't have better schools\" (i.e., event). However, \"can't have better schools\" as [C] is incorrect because it is an event instead of the entity of \"schools\" which already makes sense.\nAlso, \"raising taxes\" as [A] is incorrect because the placeholder is not taken directly from the text. Here \"raising taxes\" is chosen as [A] but the word \"raising\" does not appear in the input text. Therefore the correct correct choice for [A] is \"raise taxes\".\n\nAgain, please only select the placeholders directly from the text!\n", # instruction
        "We either have to cut taxes or leave a huge debt for our children.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth: Fast Mistral patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


['<s> You are a master at identifying the structure of logic fallacies\n\n### Instruction:\n# Task\nIdentify the underlying structure of a fallacious argument which is a False Dilemma.\nGiven a list of fallacy templates, your task is to choose a template that best describes the underlying fallacy structure, choosing the template\'s placeholders, [A] and [C], directly from the input text. Additionally, the text must be a consecutive sequence of one or more terms without any conjugation. Please follow the output format.\n\n# Definitions\nEntity: a noun phrase in the input.\nEvent: a verb phrase in the input.\nPlaceholder: A fill-in-the-blank choice within a template. Each placeholder may either be an entity or an event.\nPlease note! Placeholders can ONLY be either an entity (i.e., noun phrase) or an event (i.e., verb phrase) and may not be any other type of phrase (e.g., prepositional phrase).\n\n# List of Templates\nTemplate No.1:\nPremise 1: An entity/event [A] promotes a good entity/

In [14]:
import pandas as pd

df = pd.read_json('other_dataset.json')

In [15]:
df

,fallacy,instruction,input,output,system
0,Faulty Generalization,# Task\nIdentify the underlying structure of a...,"No, because if you start with same sex marriag...",Template No.=5\n[A]=\n[C]=\n[A']=\n[C']=\n,You are a master at identifying the structure ...
1,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Temelin exploded. It is never ok to use it.,Template No.=5\n[A]=\n[C]=\n[A']=\n[C']=\n,You are a master at identifying the structure ...
2,Faulty Generalization,# Task\nIdentify the underlying structure of a...,If you fight once you will never stop fighting...,Template No.=5\n[A]=\n[C]=\n[A']=\n[C']=\n,You are a master at identifying the structure ...
3,Faulty Generalization,# Task\nIdentify the underlying structure of a...,My son has developed anxiety because of the co...,Template No.=1\n[A]=continuous grading\n[C]=al...,You are a master at identifying the structure ...
4,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Some people did already die because of medicin...,Template No.=1\n[A]=such medicine\n[C]=all oth...,You are a master at identifying the structure ...
5,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Hitler censored media. Kim yong il censored me...,Template No.=1\n[A]=only bad people\n[C]=media...,You are a master at identifying the structure ...
6,Faulty Generalization,# Task\nIdentify the underlying structure of a...,In my region 40 % of the population is smoking...,Template No.=1\n[A]=smoking\n[C]=non-smokers\n...,You are a master at identifying the structure ...
7,Faulty Generalization,# Task\nIdentify the underlying structure of a...,"All TV-Shows are bad. Look at ""the bachelor"". ...",Template No.=1\n[A]=all TV-Shows\n[C]=learn fr...,You are a master at identifying the structure ...
8,Faulty Generalization,# Task\nIdentify the underlying structure of a...,School detention in high school means more sch...,Template No.=1\n[A]=school detention\n[C]=scho...,You are a master at identifying the structure ...
9,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Uniforms have shown to be useful and well-acce...,Template No.=3\n[A]=uniforms\n[C]=all schools\...,You are a master at identifying the structure ...


In [16]:
fallacy = df['fallacy'].to_list()
dev_instruction = df['instruction'].to_list()
input = df['input'].to_list()
gold = df['output'].to_list()

In [17]:
from transformers import TextStreamer, AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Assuming you have already defined your model and tokenizer
# model = AutoModelForCausalLM.from_pretrained('your-model-name')
# tokenizer = AutoTokenizer.from_pretrained('your-model-name')
# model.to('cuda')


outputs = []

for inst, input_text in zip(dev_instruction, input):
    prompt = alpaca_prompt.format(inst, input_text,"")
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate output
    output_sequences = model.generate(
        **inputs,
        max_new_tokens=64
    )

    # Decode the generated tokens
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Extract the generated part after the "Output: " prompt
    generated_output = generated_text.split("Response: ")[-1].strip()

    # Append to the outputs list
    outputs.append(generated_output)


In [18]:
# Create a DataFrame with instructions, inputs, and outputs
result_df = pd.DataFrame({
    'fallacy': fallacy,
    'instruction': dev_instruction,
    'input': input,
    'gold_label' : gold,
    'output': outputs
})

# Optional: Save to a CSV file
#result_df.to_csv('generated_outputs.csv', index=False)

In [20]:
import re
output_temp = []
for i in range (60):
  if result_df['fallacy'][i] == 'False Dilemma' or result_df['fallacy'][i] == 'False Causality':
    pattern = r'Response:\s*(Template No\.\=\d+)\n(\[A\]\=.*?)\n(\[C\]\=.*?)(?:\n|$)'
    match = re.search(pattern, result_df['output'][i], re.DOTALL)

    if match:
      response_text = match.group(1) + '\n' + match.group(2) + '\n' + match.group(3)

    output_temp.append(response_text)

  if result_df['fallacy'][i] == 'Faulty Generalization':
    pattern = r'Response:\s*(Template No\.\=\d+)\n(\[A\]\=.*?)\n(\[C\]\=.*?)(\[A\'\]\=.*?)(\[C\'\]\=.*?)(?:\n|$)'
    match = re.search(pattern, result_df['output'][i], re.DOTALL)

    if match:
      response_text = match.group(1) + '\n' + match.group(2) + '\n' + match.group(3) + '\n' + match.group(4) + '\n' + match.group(5)

    output_temp.append(response_text)

  if result_df['fallacy'][i] == 'Fallacy of Credibility':
    pattern = r'Response:\s*(Template No\.\=\d+)\n(\[A\]\=.*?)\n(\[C\]\=.*?)(\[X\]\=.*?)(?:\n|$)'
    match = re.search(pattern, result_df['output'][i], re.DOTALL)

    if match:
      response_text = match.group(1) + '\n' + match.group(2) + '\n' + match.group(3) + '\n' + match.group(4)

    output_temp.append(response_text)

In [21]:
result_df.drop(columns=['output'], inplace=True)
result_df['output'] = output_temp

In [22]:
result_df

,fallacy,instruction,input,gold_label,output
0,Faulty Generalization,# Task\nIdentify the underlying structure of a...,"No, because if you start with same sex marriag...",Template No.=5\n[A]=\n[C]=\n[A']=\n[C']=\n,Template No.=5\n[A]=\n[C]=\n\n[A']=\n\n[C']=
1,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Temelin exploded. It is never ok to use it.,Template No.=5\n[A]=\n[C]=\n[A']=\n[C']=\n,Template No.=5\n[A]=\n[C]=\n\n[A']=\n\n[C']=
2,Faulty Generalization,# Task\nIdentify the underlying structure of a...,If you fight once you will never stop fighting...,Template No.=5\n[A]=\n[C]=\n[A']=\n[C']=\n,Template No.=2\n[A]=fight\n[C]=never stop figh...
3,Faulty Generalization,# Task\nIdentify the underlying structure of a...,My son has developed anxiety because of the co...,Template No.=1\n[A]=continuous grading\n[C]=al...,Template No.=2\n[A]=continuous grading\n[C]=an...
4,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Some people did already die because of medicin...,Template No.=1\n[A]=such medicine\n[C]=all oth...,Template No.=2\n[A]=take such medicine\n[C]=al...
5,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Hitler censored media. Kim yong il censored me...,Template No.=1\n[A]=only bad people\n[C]=media...,Template No.=5\n[A]=\n[C]=\n\n[A']=\n\n[C']=
6,Faulty Generalization,# Task\nIdentify the underlying structure of a...,In my region 40 % of the population is smoking...,Template No.=1\n[A]=smoking\n[C]=non-smokers\n...,Template No.=3\n[A]=protect the non-smokers\n[...
7,Faulty Generalization,# Task\nIdentify the underlying structure of a...,"All TV-Shows are bad. Look at ""the bachelor"". ...",Template No.=1\n[A]=all TV-Shows\n[C]=learn fr...,Template No.=2\n[A]=all TV-Shows\n[C]=children...
8,Faulty Generalization,# Task\nIdentify the underlying structure of a...,School detention in high school means more sch...,Template No.=1\n[A]=school detention\n[C]=scho...,Template No.=2\n[A]=school detention in high s...
9,Faulty Generalization,# Task\nIdentify the underlying structure of a...,Uniforms have shown to be useful and well-acce...,Template No.=3\n[A]=uniforms\n[C]=all schools\...,Template No.=3\n[A]=introduce them\n[C]=school...


In [23]:
result_df.to_csv('other_outputs.csv', index=False)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>